# **Setting up environment**

In [ ]:
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
import cv2 as cv
import pickle
import math
from collections import Counter

# **User-defined functions**

In [ ]:
def computation_descriptor(image):
    h = [1, -3, 3, -1]
    t, q = 2, 3
    for row in range(image.shape[0]):
        for col in range(image.shape[1]-3):
            temp = image[row][col] + (-3*image[row][col+1]) + (3*image[row][col+2]) + (-1*image[row][col+3])
            image[row][col] = min(t, max(-t, math.floor(image[row][col]/q)))
    
    
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            if (image[row][col] > 2):
                image[row][col] = 0
                
    return image

In [ ]:
def window(image):
    windows = []
    stepsize = 8
    for row in range(0, image.shape[1]-128, stepsize):
        for col in range(0, image.shape[0]-128, stepsize):
            windows.append(image[row:row+128,col:col+128])
            
    return windows

In [ ]:

def makeHist(image):
    dict = {}
    count = Counter()
    for i in range(image.shape[1]-3):
        for j in image[:, i : i+4]:
            if tuple(j) in count:
                count[tuple(j)] += 1
            elif tuple(j)[::-1] in count:
                count[tuple(j)[::-1]] += 1
            else :
                count[tuple(j)] = 1
    
    return count.values()

In [ ]:
def makeList325(lis):
    listsize = len(lis)
    x= np.zeros(325-listsize).astype('int')
    return lis + list(x)

In [ ]:
def computeStatisticalDescriptor(frames):
    rhist=[]
    chist=[]
    for i in range(5,8):
        rowResidual = frames[i].astype('int')
        rowResidual = computation_descriptor(rowResidual)
        row_window = window(rowResidual)
        for k in range(len(row_window)):
            rhist.append(makeList325(list(makeHist(row_window[k]))))

        
        colResidual = frames[i].T.astype('int')
        colResidual = computation_descriptor(colResidual)
        col_window = window(colResidual)
        for k in range(len(col_window)):
            chist.append(makeList325(list(makeHist(col_window[k]))))

    return rhist,chist

In [ ]:
def opticalFlow(image):
    angle = []
    magnitude=[]
    prev = image[4]
    for i in range(5, 8):
        flow = cv.calcOpticalFlowFarneback(prev, image[i], None,0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
        angle.append(ang)
        magnitude.append(mag)
    
    return np.array(magnitude),np.array(angle)

In [ ]:
def normalize(mag,ang):
    for i in range(len(mag)):
        mag[i]=cv.normalize(mag[i],None,0,255,cv.NORM_MINMAX)
        ang[i] =ang[i]*360/np.pi/2
    return mag,ang

In [ ]:
def computeMagAngle(mag,ang):
    ohist=[]
    print (len (mag))
    for i in range(len(mag)):
        print(i , end = "")
        magwindow = window(mag[i])
        angwindow= window(ang[i])
        binhist=[]
        for k in range(len(magwindow)):
            I1=magwindow[k]
            I2=angwindow[k]
            h=I1.shape[0]
            w=I1.shape[1]
            x=0
            y=0
            bin_array=np.zeros((4,4),dtype=int);
            for i in range(h):
                for j in range(w):
                    x=int(I1[i][j]//64)
                    if(x==4):
                        x=3
                    y=int(I2[i][j]//90)
                    if(y==4):
                        y=3
                    bin_array[x][y]+=1
            bin_array=np.reshape(bin_array,(1,16))
            bin_array=bin_array[0]
            binhist.append(bin_array)
        ohist.append(binhist)
    return ohist

In [ ]:
def computeFinalhistogram(rhist,chist,ohist):
    rowh = []
    colh = []
    for i in range(0,len(rhist),len(rhist)//3):
        rowh.append(rhist[i:i+4480])
        colh.append(chist[i:i+4480])

    fhist=list()
    for ind in range(3):
        fhist.append(np.concatenate([np.array(rowh[ind]),np.array(colh[ind]) , np.array(ohist[ind])] ,axis=1))

    return fhist

In [ ]:
def frameCapture(path):
    frames=[]
    vidObj = cv.VideoCapture(path)
    count = 0
    success = 1
    while count<30:
        success, image = vidObj.read()
        frames.append(cv.cvtColor(image,cv.COLOR_BGR2GRAY))
        count += 1
    return frames

In [ ]:
def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = np.linalg.pinv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

# **Results**

## **Reading the video files**

In [ ]:
# For original video
orig_frame = frameCapture("Original.avi")
# For tempered video - 1
temp_frame = frameCapture("Tempered_1.avi")
# For tempered video - 2
temp2_frame = frameCapture("Tempered_2.avi")
# For tempered video - 3
temp3_frame = frameCapture("Tempered_3.avi")

## **Computing Statistical Descriptor**

In [ ]:
rhist, chist = computeStatisticalDescriptor(orig_frame)
trhist,tchist = computeStatisticalDescriptor(temp_frame)
t2rhist, t2chist = computeStatisticalDescriptor(temp2_frame)
t3rhist, t3chist = computeStatisticalDescriptor(temp3_frame)

## **Calculating optical flow**

In [ ]:
# For original video
mag,ang = opticalFlow(orig_frame)
mag,ang = normalize(mag,ang)
ohist = computeMagAngle(mag,ang)
# For tempered video - 1
tmag,tang = opticalFlow(temp_frame)
tmag,tang = normalize(tmag,tang)
tohist = computeMagAngle(tmag,tang)
# For tempered video - 2
mag,ang = opticalFlow(temp2_frame)
mag,ang = normalize(mag,ang)
t2ohist = computeMagAngle(mag,ang)
# For tempered video - 3
mag,ang = opticalFlow(temp3_frame)
mag,ang = normalize(mag,ang)
t3ohist = computeMagAngle(mag,ang)

## **Computing final histogram**

In [ ]:
# For original video
fhist = computeFinalhistogram(rhist,chist,ohist)
# For tempered video - 1
tfhist = computeFinalhistogram(trhist,tchist,tohist)
# For tempered video - 2
t2fhist = computeFinalhistogram(t2rhist,t2chist,t2ohist)
# For tempered video - 3
t3fhist = computeFinalhistogram(t3rhist,t3chist,t3ohist)

## **Generating dataset**

In [ ]:
# Difference in original and tempered frames
frame = orig_frame[5]-temp_frame[5]
plt.imshow(frame, interpolation='nearest')
plt.show()

### **Importing extra libraries**

In [ ]:
import pickle

### **Saving dataset**

In [ ]:
# For Original Dataset
file = open('original/rhist',"wb")
pickle.dump(rhist, file)
file.close()
file = open('original/chist',"wb")
pickle.dump(chist, file)
file.close()
file = open('original/ohist',"wb")
pickle.dump(ohist, file)
file.close()
file = open('original/fhist',"wb")
pickle.dump(fhist, file)
file.close()

file = open('original/rhist',"rb")
rhist = pickle.load(file)
file.close()
file = open('original/chist',"rb")
chist = pickle.load(file)
file.close()
file = open('original/ohist',"rb")
ohist = pickle.load(file)
file.close()
file = open('original/fhist',"rb")
fhist = pickle.load(file)
file.close()

df=pd.DataFrame(data=np.array(fhist[0]))
df.head()

# For Tempered dataset - 1
file = open('Tempered_1/rhist',"wb")
pickle.dump(trhist, file)
file.close()
file = open('Tempered_1/chist',"wb")
pickle.dump(tchist, file)
file.close()
file = open('Tempered_1/ohist',"wb")
pickle.dump(tohist, file)
file.close()
file = open('Tempered_1/fhist',"wb")
pickle.dump(tfhist, file)
file.close()

file = open('Tempered_1/rhist',"rb")
trhist = pickle.load(file)
file.close()
file = open('Tempered_1/chist',"rb")
tchist = pickle.load(file)
file.close()
file = open('Tempered_1/ohist',"rb")
tohist = pickle.load(file)
file.close()
file = open('Tempered_1/fhist',"rb")
tfhist = pickle.load(file)
file.close()

df2=pd.DataFrame(data=np.array(tfhist[0]))
df2.head()

## **Calculating Mahalanobis Distance**

In [ ]:
x=[]
for i in range(3):
    df=pd.DataFrame(data=np.array(fhist[i]))
    mah=mahalanobis(x=df2,data=df)
    x.append(mah)

norm = np.array(x)/np.max(np.array(x))
norm=norm*255
np.max(norm)
norm.shape

In [ ]:
# Interpolating the frames
max=180
index =0
w=[]
for j in range(len(norm[1])):
    if norm[1][j] > 180:
        w.append(j)

oframe6 = orig_frame[6]
tframe6 = temp_frame[6]
owindows = window(oframe6)
twindows = window(tframe6)
for i in w:
    frame = owindows[i] - twindows[i]
    xyz= np.concatenate([owindows[i] , twindows[i],frame],axis=1)
    plt.imshow(xyz,interpolation='nearest')
    plt.show()        